For this project, I will show the process of building a model to predict readmission for diabetes patients. And the process can be divided into 5 parts.

data exploration

feature engineerin

building training/validation/test samples

modelling

model evaluation

dara source:https://archive.ics.uci.edu/ml/datasets/diabetes+130-us+hospitals+for+years+1999-2008

In [ ]:
#import the library
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [ ]:
#load the data
df = pd.read_csv('diabetic_data.csv')

### check the dataset

In [ ]:
print('Number of samples:',len(df))

In [ ]:
df.info()

This data have more than 100000 items and 51 features

Then check the values of data

In [ ]:
df.head()

### feature engineering

In [ ]:
#Replace missing values
df = df.replace("?",np.nan)

In [ ]:
# count the number of rows for each type
df.groupby('readmitted').size()

This is an imbalanced dataset, and we'll deal with that later

In [ ]:
#Check the patient's discharge directions
df.groupby('discharge_disposition_id').size()

In [ ]:
#Remove meaningless data
df = df.loc[~df.discharge_disposition_id.isin([11,13,14,19,20,21])]
df.info()

You can see many string variables. You can see the meaning of ids_mapping.csv

In [ ]:
#the fuction for checking the value of features
for c in list(df.columns):
    
    # get a list of unique values
    n = df[c].unique()
    
    # if number of unique values is less than 30, print the values. Otherwise print the number of unique values
    if len(n)<30:
        print(c)
        print(n)
    else:
        print(c + ': ' +str(len(n)) + ' unique values')

Checking for missing values

In [ ]:
# replace ? with nan
df = df.replace('?',np.nan)

In [ ]:
num_list = ['time_in_hospital','num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient','number_diagnoses']
df[num_list].isnull().sum()

In [ ]:
cat_list = ['race', 'gender', 
       'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed','payer_code','medical_specialty']
df[cat_list].isnull().sum().sort_values(ascending = False)

deal with features with missing values

In [ ]:
df['race'] = df['race'].fillna('UNK')
# too many missing values, so drop them
df = df.drop(labels=['medical_specialty','payer_code'], axis=1)

In [ ]:
df.info()

In [ ]:
#segmentation feature
df[['age', 'weight']]

In [ ]:
df.groupby('age').size()

In [ ]:
age_id = {'[0-10)':0, 
          '[10-20)':10, 
          '[20-30)':20, 
          '[30-40)':30, 
          '[40-50)':40, 
          '[50-60)':50,
          '[60-70)':60, 
          '[70-80)':70, 
          '[80-90)':80, 
          '[90-100)':90}
df['age_group'] = df.age.replace(age_id)

In [ ]:
#too many missing values
df = df.drop(labels=['age','weight'], axis=1)

Considering the clinical information, these data are presented without clinical significance

In [ ]:
drop_list = ['examide' , 'citoglipton','encounter_id','patient_nbr']  
df.drop(drop_list,axis=1, inplace=True)

In [ ]:
# deal with missiing value
diag_list = ['diag_1','diag_2','diag_3']

for col in diag_list:
    df[col].fillna('NaN', inplace=True)

In [ ]:
import re

# feature eigineering for dig1,dig2,dig3
def transformFunc(value):
    value = re.sub("V[0-9]*", "0", value) # V 
    value = re.sub("E[0-9]*", "0", value) # E 
    value = re.sub('NaN', "-1", value) # Nan 
    return value

def transformCategory(value):
    if value>=390 and value<=459 or value==785:
        category = 'Circulatory'
    elif value>=460 and value<=519 or value==786:
        category = 'Respiratory'
    elif value>=520 and value<=579 or value==787:
        category = 'Digestive'
    elif value==250:
        category = 'Diabetes'
    elif value>=800 and value<=999:
        category = 'Injury'          
    elif value>=710 and value<=739:
        category = 'Musculoskeletal'   
    elif value>=580 and value<=629 or value==788:
        category = 'Genitourinary'
    elif value>=140 and value<=239 :
        category = 'Neoplasms'
    elif value==-1:
        category = 'NAN'
    else :
        category = 'Other'
    return category

for col in diag_list:
    df[col] = df[col].apply(transformFunc)
    df[col] = df[col].astype(float)

for col in diag_list:
    df[col] = df[col].apply(transformCategory)

In [ ]:
# encoding for the 24 Drug 
drugs = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'glipizide', 'glyburide', 'pioglitazone',
        'rosiglitazone', 'acarbose', 'miglitol', 'insulin', 'glyburide-metformin', 'tolazamide', 'metformin-pioglitazone',
        'metformin-rosiglitazone', 'glimepiride-pioglitazone', 'glipizide-metformin', 'troglitazone', 'tolbutamide', 'acetohexamide']

for col in drugs:
    df[col] = df[col].replace(['No','Steady','Up','Down'],[0,1,1,1])
    df[col] = df[col].astype(int)

In [ ]:
# deal with A1Cresult and max_glu_serum
df['A1Cresult'] = df['A1Cresult'].replace(['>7','>8','Norm','None'],[1,1,0,-99])
df['max_glu_serum'] = df['max_glu_serum'].replace(['>200','>300','Norm','None'],[1,1,0,-99])

In [ ]:
df[list(df.columns)[:10]].head()
df[list(df.columns)[10:20]].head()
df[list(df.columns)[20:30]].head()
df[list(df.columns)[30:40]].head()
df[list(df.columns)[40:]].head()

In [ ]:
#one-hot encoding
one_hot_data = pd.get_dummies(df, columns=['race'], prefix=["enc"])
columns_ids = ['admission_type_id', 'discharge_disposition_id', 'admission_source_id']
one_hot_data[columns_ids] = df[columns_ids].astype('str')
one_hot_data = pd.get_dummies(one_hot_data, columns=columns_ids)
data=one_hot_data.copy()
data.readmitted = [1 if each=='<30' else 0 for each in data.readmitted]

In [ ]:
# Encode the label feature

from sklearn.preprocessing import LabelEncoder
for col in diag_list:
    label_enc = LabelEncoder()
    data[col] = label_enc.fit_transform(data[col])
    
    
binary = ['change', 'diabetesMed', 'gender']
from category_encoders import BinaryEncoder
binary_enc = BinaryEncoder(cols=binary)
binary_enc.fit_transform(data)
data = binary_enc.fit_transform(data)

In [ ]:
# split training/validation/test samples
X = data.drop(columns="readmitted", axis=1)
Y = data.readmitted
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25)

In [ ]:
# deal with imblanced data by using SMOTE function
from imblearn.over_sampling import SMOTE
column_name = X_train.columns
X_train,y_train = SMOTE().fit_sample(X_train,y_train)
X_train = pd.DataFrame(X_train,columns = column_name)
y_train.value_counts()

In [ ]:
# The generic code that outputs the modeling results comes from Github
from sklearn.metrics import confusion_matrix, accuracy_score,f1_score,recall_score,mean_squared_error, r2_score, roc_auc_score, roc_curve, classification_report
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score,f1_score
from sklearn.metrics import confusion_matrix as cm

def calc_specificity(y_actual, y_pred, thresh):
    # calculates specificity
    return sum((y_pred < thresh) & (y_actual == 0)) /sum(y_actual ==0)

def print_report(y_actual, y_pred, thresh):
    
    auc = roc_auc_score(y_actual, y_pred)
    accuracy = accuracy_score(y_actual, (y_pred > thresh))
    recall = recall_score(y_actual, (y_pred > thresh))
    precision = precision_score(y_actual, (y_pred > thresh))
    fscore = f1_score(y_actual,(y_pred > thresh) )
    specificity = calc_specificity(y_actual, y_pred, thresh)
    print('AUC:%.3f'%auc)
    print('accuracy:%.3f'%accuracy)
    print('recall:%.3f'%recall)
    print('precision:%.3f'%precision)
    print('fscore:%.3f'%fscore)
    print('specificity:%.3f'%specificity)
    print(' ')
    return auc, accuracy, recall, precision,fscore, specificity

SVM modelling and evaluation

In [ ]:
from sklearn.svm import LinearSVC
def run_SVM(X_train,X_test):
    svm = LinearSVC()
    svm.fit(X_train, y_train)
    svm_pred = svm.predict(X_test)
    svm_pred_prob = svm.decision_function(X_test)
    svm_accuracy = sklearn.metrics.accuracy_score(y_test, svm_pred)
    print("Accuracy : ",svm_accuracy)
    k = pd.DataFrame(sklearn.metrics.confusion_matrix(y_test,svm_pred))
    print(k)
    
    
    predictions = svm.predict(X_train)
    train_score = round(accuracy_score(y_train, predictions), 3)
    cm_train = cm(y_train, predictions)

    predictions = svm.predict(X_test)
    val_score = round(accuracy_score(y_test, predictions), 3)
    cm_val = cm(y_test, predictions)

    fig, (ax1,ax2) = plt.subplots(nrows=1,ncols=2,figsize=(15,5)) 
    sns.heatmap(cm_train, annot=True, fmt=".0f",ax=ax1)
    ax1.set_xlabel('Predicted Values')
    ax1.set_ylabel('Actual Values')
    ax1.set_title('Train Accuracy Score: {0}'.format(train_score), size = 15)
    sns.heatmap(cm_val, annot=True, fmt=".0f",ax=ax2)
    ax2.set_xlabel('Predicted Values')
    ax2.set_ylabel('Actual Values')
    ax2.set_title('Validation Accuracy Score: {0}'.format(val_score), size = 15)
    
    plt.show()
    
    
    
    
    return svm_pred_prob

SVM_prob = run_SVM(X_train,X_test)

LR modelling and evaluation

In [ ]:
from sklearn.linear_model import LogisticRegression
def run_lg(X_train,X_test):
    lg = LogisticRegression(C=1)
    lg.fit(X_train,y_train)
    lg_pred = lg.predict(X_test)
    lg_pred_prob = lg.predict_proba(X_test)[:,1]
    k = pd.DataFrame(sklearn.metrics.confusion_matrix(y_test,lg_pred))
    lg_accuracy = sklearn.metrics.accuracy_score(y_test, lg_pred)
    print("Accuracy : ",lg_accuracy)
    k = pd.DataFrame(sklearn.metrics.confusion_matrix(y_test,lg_pred))
    print(k)
    
    
    predictions = lg.predict(X_train)
    train_score = round(accuracy_score(y_train, predictions), 3)
    cm_train = cm(y_train, predictions)

    predictions = lg.predict(X_test)
    val_score = round(accuracy_score(y_test, predictions), 3)
    cm_val = cm(y_test, predictions)

    fig, (ax1,ax2) = plt.subplots(nrows=1,ncols=2,figsize=(15,5)) 
    sns.heatmap(cm_train, annot=True, fmt=".0f",ax=ax1)
    ax1.set_xlabel('Predicted Values')
    ax1.set_ylabel('Actual Values')
    ax1.set_title('Train Accuracy Score: {0}'.format(train_score), size = 15)
    sns.heatmap(cm_val, annot=True, fmt=".0f",ax=ax2)
    ax2.set_xlabel('Predicted Values')
    ax2.set_ylabel('Actual Values')
    ax2.set_title('Validation Accuracy Score: {0}'.format(val_score), size = 15)
    
    plt.show()
    
    
    return lg_pred_prob

lg_prob = run_lg(X_train,X_test)

In [ ]:
Xgboost modelling and evaluation

In [ ]:
import xgboost

def run_xgb(X_train,X_test):
    xgb= xgboost.XGBClassifier(n_estimators =200,max_depth =8,learning_rate = 0.01)

    xgb.fit(X_train, y_train)

    xgb_pred = xgb.predict(X_test) 
    xgb_pred_prob = xgb.predict_proba(X_test)[:,1]

    xgb_accuracy = sklearn.metrics.accuracy_score(y_test, xgb_pred)
    xgb_roc = sklearn.metrics.roc_auc_score(y_test, xgb_pred_prob)
    print("Accuracy : ",xgb_accuracy)
    print("ROC Score : ",xgb_roc)
    k = pd.DataFrame(sklearn.metrics.confusion_matrix(y_test,xgb_pred))
    print(k)
    
    
    from xgboost import plot_importance
    plot_importance(xgb,max_num_features=10)
    plt.savefig("./xgboost.jpg")
    plt.show()
    
    
    return xgb_pred_prob
xgb_prob = run_xgb(X_train,X_test)

In [ ]:
Lightgbm modelling and evaluation

In [ ]:
from lightgbm import LGBMClassifier
import lightgbm

def run_lgb(X_train,X_test):
    lgb= LGBMClassifier(num_leaves =63,max_depth =10,learning_rate = 0.001)

    lgb.fit(X_train, y_train)

    lgb_pred = lgb.predict(X_test) 
    lgb_pred_prob = lgb.predict_proba(X_test)[:,1]

    lgb_accuracy = sklearn.metrics.accuracy_score(y_test, lgb_pred)
    lgb_roc = sklearn.metrics.roc_auc_score(y_test, lgb_pred_prob)
    print("Accuracy : ",lgb_accuracy)
    print("ROC Score : ",lgb_roc)
    k = pd.DataFrame(sklearn.metrics.confusion_matrix(y_test,lgb_pred))
    print(k)
    
#     predictions = lgb.predict(X_train)
#     train_score = round(accuracy_score(y_train, predictions), 3)
#     cm_train = cm(y_train, predictions)

#     predictions = lgb.predict(X_test)
#     val_score = round(accuracy_score(y_test, predictions), 3)
#     cm_val = cm(y_test, predictions)
#     print(accuracy_score(y_test, predictions))

#     fig, (ax1,ax2) = plt.subplots(nrows=1,ncols=2,figsize=(15,5)) 
#     sns.heatmap(cm_train, annot=True, fmt=".0f",ax=ax1)
#     ax1.set_xlabel('Predicted Values')
#     ax1.set_ylabel('Actual Values')
#     ax1.set_title('Train Accuracy Score: {0}'.format(train_score), size = 15)
#     sns.heatmap(cm_val, annot=True, fmt=".0f",ax=ax2)
#     ax2.set_xlabel('Predicted Values')
#     ax2.set_ylabel('Actual Values')
#     ax2.set_title('Validation Accuracy Score: {0}'.format(val_score), size = 15)
#     plt.show()

#     print(type(cm_train))
    
#     ax=lightgbm.plot_importance(lgb, max_num_features=10)
#     plt.show()
#     plt.savefig("./lightgbm1.jpg")
    
    
    plt.figure(figsize=(30,12))
    lightgbm.plot_importance(lgb, max_num_features=10)
    plt.savefig("./lightgbm2.jpg")
    plt.show()
    
    
    
    
    return lgb_pred_prob
lgb_prob = run_lgb(X_train,X_test)

In [ ]:
merge = {
  'SVM_prob': SVM_prob,
  'lg_prob': lg_prob,
  'xgb_prob': xgb_prob,
  'lgb_prob': lgb_prob,
    
  }  
models=pd.DataFrame(merge)

w=[0.2,0.1,0.35,0.35]
w_average = [1/5,1/5,1/5,1/5]
direct=np.dot(models.values,w)

stacking_pred =[1 if i>0.5 else 0 for i in direct] 
nn_accuracy = sklearn.metrics.accuracy_score(y_test, stacking_pred)
nn_roc = sklearn.metrics.roc_auc_score(y_test, direct)
print("Accuracy : ",nn_accuracy)
print("ROC Score : ",nn_roc)
k = pd.DataFrame(sklearn.metrics.confusion_matrix(y_test,stacking_pred))